# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [10]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,60.086593,0.647887,0.467428,0.033113,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.977558,0.975741,0.969452,0.971249,0.974491,0.003538,1
11,41.926287,1.100187,0.346295,0.048726,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.977558,0.974843,0.968553,0.970350,0.973773,0.003708,2
7,36.292582,2.806112,0.576939,0.066512,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.974865,0.975763,0.974843,0.965858,0.972147,0.972695,0.003627,3
10,30.173296,1.256649,0.403678,0.021329,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975763,0.974865,0.973046,0.968553,0.969452,0.972336,0.002873,4
5,48.656202,0.473787,0.717105,0.070479,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.976661,0.974865,0.973944,0.964061,0.969452,0.971797,0.004539,5


In [13]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,38.829272,0.695652,0.276310,0.028044,None,300,"{'max_depth': None, 'n_estimators': 300}",0.979354,0.974865,0.973944,0.967655,0.972147,0.973593,0.003802,1
8,50.035461,0.876464,0.450060,0.021007,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978456,0.974865,0.974843,0.965858,0.972147,0.973234,0.004198,2
7,26.516635,0.310189,0.538428,0.107542,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.975763,0.973944,0.966757,0.971249,0.973234,0.004002,3
10,27.185346,0.557272,0.356165,0.054323,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.974865,0.973046,0.968553,0.969452,0.972695,0.003351,4
3,5.741907,3.046098,1.236656,1.056329,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.972172,0.974865,0.972147,0.965858,0.970350,0.971079,0.002982,5
